# Image collections loading reference


## Description

Use this notebook as a reference for how to load Landsat,  and Sentinel-2 optical image collections

## Load packages

Import Python packages that are used for the analysis.


In [ ]:
%matplotlib inline

import geemap as gmap
import ee

### Connect to Google Earth Engine (GEE)

In [ ]:
m = gmap.Map()

***

## How to load a Sentinel 2 image collection

Use this product:

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#description

In [ ]:
#Define a function for cloud masking and recaling Sentinel 2 images
def cloudMaskSentinel2(image):
    
    #select the quality control band
    qa = image.select('QA60')

    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    #rescale the image
    image = image.updateMask(mask).divide(10000)
    
    return image

### Load sentinel-2 and filter by a location and date

In [ ]:
Map = gmap.Map(center=[-35.2041, 149.2721], zoom=9)

# Create a point with set coordinates (canberra parliament)
point = ee.Geometry.Point([149.1244, -35.3096])
    
#load the harmonized S2 product
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterDate('2019-12-01', '2019-12-31') #filter dates
                  .filterBounds(point) #filter to some point
     )

# mask the clouds and rescale
s2_masked = s2.map(cloudMaskSentinel2)

#before scaling the numbers are 0-10000
vis_unscaled = {
  'min': 0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
};

#after scaling, numbers are 0-1
vis_scaled = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
};

# add just the first image in the collection to the map
Map.addLayer(s2.first(), vis_unscaled, 'Sentinel-2 RGB')
Map.addLayer(s2_masked.first(), vis_scaled, 'Sentinel-2 RGB masked')
Map.addLayerControl()
Map

## How to load a Landsat image collection

Use this product (adjust depending on whether you want to load Landsat 5,7,8 or 9):

https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2

In [ ]:
def rescale_landsatC2(image):
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True)

def cloudMaskL8Collection2(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

  # Replace the original bands with the scaled ones and apply the masks.
    return image.updateMask(qaMask) \
          .updateMask(saturationMask)

### Load Landsat 8 and filter by a location and date

In [ ]:
# We give the 'center' location, and a 'zoom' level.
Map = gmap.Map(center=[-35.3096, 149.1244], zoom=9)

# Create a point with set coordinates
point = ee.Geometry.Point([149.1244, -35.3096])

# Search for a landsat 8 image collection 2
ls = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            .filterDate('2023-07', '2023-08') #filter by images in July 2023
            .filterBounds(point)  # filter to images around parliament
            .map(rescale_landsatC2)  # map the rescaling function
     )

#now mask the clouds
ls_masked = ls.map(cloudMaskL8Collection2)

# Now we select the bands we want to display for the collection 2 image
vis_ls = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
              'min': 0,
              'max': 0.4}

# Add only the "first" image of landsat 8 image to our map
Map.addLayer(ls.first(), vis_ls, 'Unmasked Landsat') 
Map.addLayer(ls_masked.first(), vis_ls, 'Masked Landsat')
Map.addLayerControl()
Map

***